In [53]:
import pandas as pd
import yfinance as yf
import requests
from datetime import datetime
from azure.storage.blob import BlobServiceClient
from io import StringIO
from pprint import pprint



In [18]:
AZURE_STORAGE_ACCOUNT = "cpaastask"
AZURE_CONTAINER = "analytics"
AZURE_SAS_TOKEN = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-10-15T04:51:35Z&st=2025-09-24T20:36:35Z&spr=https&sig=U13SlWdeg4PddtSHGC%2B7VjVMMDARjP49MBhqF6yNXw8%3D"

account_url = f"https://{AZURE_STORAGE_ACCOUNT}.blob.core.windows.net"
blob_service = BlobServiceClient(account_url=account_url, credential=AZURE_SAS_TOKEN)
container = blob_service.get_container_client(AZURE_CONTAINER)

if container: print("Connection established")

Connection established


In [20]:
# RAW Alpha Vantage call

ALPHA_VANTAGE_API_KEY = "ARZ8YPRK3FZXLBSL"
BASE_URL = "https://www.alphavantage.co/query"
params = {
    "function": "TIME_SERIES_DAILY",  
    "symbol": "TWLO",
    "outputsize": "full",
    "apikey": ALPHA_VANTAGE_API_KEY,
}

resp = requests.get(BASE_URL, params=params, timeout=60)
print("HTTP status:", resp.status_code)
data = resp.json()

pprint(data)


HTTP status: 200
{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and '
                                 'Volumes',
               '2. Symbol': 'TWLO',
               '3. Last Refreshed': '2025-09-25',
               '4. Output Size': 'Full size',
               '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2016-06-23': {'1. open': '23.9900',
                                        '2. high': '29.6100',
                                        '3. low': '23.6600',
                                        '4. close': '28.7900',
                                        '5. volume': '21299009'},
                         '2016-06-24': {'1. open': '27.5400',
                                        '2. high': '28.7390',
                                        '3. low': '26.0500',
                                        '4. close': '26.3000',
                                        '5. volume': '4670689'},
                         '2016-06-27': {'1. open': '27.3

In [45]:
def fetch_daily_from_date(symbol: str, start_date: str = "2020-01-01"):
    """Return a dataframe with daily prices from start_date onward (YYYY-MM-DD)."""
    try:
        print(f"Fetching {symbol} from Alpha Vantage...")

        params = {
            "function": "TIME_SERIES_DAILY",
            "symbol": symbol,
            "outputsize": "full",   # full = complete history
            "apikey": ALPHA_VANTAGE_API_KEY,
        }

        resp = requests.get(BASE_URL, params=params, timeout=60)
        print(f"HTTP status: {resp.status_code}")
        resp.raise_for_status()

        data = resp.json()

        if "Time Series (Daily)" not in data:
            print("Unexpected response format")
            print(data)
            return pd.DataFrame()  # return empty dataframe

        # Convert dict to DataFrame
        df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient="index")
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()

        # Filter from start_date
        df = df[df.index >= pd.to_datetime(start_date)]

        # Convert to numeric
        df = df.astype(float)

        # Reset index -> "date" column
        df = df.reset_index().rename(columns={"index": "date"})
        df["company"] = symbol

        print(f"Retrieved {len(df)} rows for {symbol} (from {df['date'].min().date()} to {df['date'].max().date()})")

        return df

    except Exception as e:
        print(f"Error fetching {symbol}: {e}")
        return pd.DataFrame()  # safe fallback


In [46]:
twlo = fetch_daily_from_date("TWLO", "2020-01-01")
twlo.head()

Fetching TWLO from Alpha Vantage...
HTTP status: 200
Retrieved 1441 rows for TWLO (from 2020-01-02 to 2025-09-25)


,date,1. open,2. high,3. low,4. close,5. volume,company
0,2020-01-02,100.01,103.15,99.66,103.15,2876697.0,TWLO
1,2020-01-03,101.10,104.55,100.76,103.52,2502762.0,TWLO
2,2020-01-06,102.74,107.74,101.77,107.46,3938663.0,TWLO
3,2020-01-07,107.63,109.94,107.37,108.06,2646387.0,TWLO
4,2020-01-08,108.80,110.82,108.20,109.39,3034975.0,TWLO


In [47]:
band = fetch_daily_from_date("BAND", "2020-01-01")
band.head()

Fetching BAND from Alpha Vantage...
HTTP status: 200
Retrieved 1441 rows for BAND (from 2020-01-02 to 2025-09-25)


,date,1. open,2. high,3. low,4. close,5. volume,company
0,2020-01-02,64.48,64.840,63.605,64.22,120663.0,BAND
1,2020-01-03,63.69,66.100,63.690,65.12,148024.0,BAND
2,2020-01-06,64.53,67.605,63.400,67.43,198237.0,BAND
3,2020-01-07,67.51,68.820,66.630,67.99,178021.0,BAND
4,2020-01-08,67.82,69.290,67.290,67.74,191619.0,BAND


In [48]:
eght = fetch_daily_from_date("EGHT", "2020-01-01")
eght.head()

Fetching EGHT from Alpha Vantage...
HTTP status: 200
Retrieved 1441 rows for EGHT (from 2020-01-02 to 2025-09-25)


,date,1. open,2. high,3. low,4. close,5. volume,company
0,2020-01-02,18.37,18.65,18.36,18.63,1622424.0,EGHT
1,2020-01-03,18.40,18.67,18.34,18.50,1066907.0,EGHT
2,2020-01-06,18.36,18.57,18.13,18.45,1031127.0,EGHT
3,2020-01-07,18.55,18.59,18.19,18.52,1332523.0,EGHT
4,2020-01-08,18.61,19.17,18.56,19.05,725545.0,EGHT


In [49]:
combined_df = pd.concat([twlo, band, eght])

In [50]:
combined_df.head()

,date,1. open,2. high,3. low,4. close,5. volume,company
0,2020-01-02,100.01,103.15,99.66,103.15,2876697.0,TWLO
1,2020-01-03,101.10,104.55,100.76,103.52,2502762.0,TWLO
2,2020-01-06,102.74,107.74,101.77,107.46,3938663.0,TWLO
3,2020-01-07,107.63,109.94,107.37,108.06,2646387.0,TWLO
4,2020-01-08,108.80,110.82,108.20,109.39,3034975.0,TWLO


In [41]:
combined_df["company"].value_counts()

company
TWLO    1441
BAND    1441
EGHT    1441
Name: count, dtype: int64

In [51]:
rename_map = {
    "1. open": "open",
    "2. high": "high",
    "3. low": "low",
    "4. close": "close",
    "5. volume": "volume",
}

combined_df = combined_df.rename(columns=rename_map)

In [52]:
combined_df.head()

,date,open,high,low,close,volume,company
0,2020-01-02,100.01,103.15,99.66,103.15,2876697.0,TWLO
1,2020-01-03,101.10,104.55,100.76,103.52,2502762.0,TWLO
2,2020-01-06,102.74,107.74,101.77,107.46,3938663.0,TWLO
3,2020-01-07,107.63,109.94,107.37,108.06,2646387.0,TWLO
4,2020-01-08,108.80,110.82,108.20,109.39,3034975.0,TWLO


In [54]:
def upload_df_to_blob(df, blob_name: str):
    try:
        # Convert DataFrame -> CSV (in-memory, no file on disk)
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False)
        csv_buffer.seek(0)

        # Get container client
        container = blob_service.get_container_client(AZURE_CONTAINER)

        # Upload (overwrite if exists)
        container.upload_blob(name=blob_name, data=csv_buffer.getvalue(), overwrite=True)

        print(f"Uploaded DataFrame to blob: {blob_name}")

    except Exception as e:
        print(f"Error uploading DataFrame: {e}")

In [56]:
upload_df_to_blob(twlo, "twlo_prices.csv")
upload_df_to_blob(eght, "eght_prices.csv")
upload_df_to_blob(band, "band_prices.csv")

Uploaded DataFrame to blob: twlo_prices.csv
Uploaded DataFrame to blob: eght_prices.csv
Uploaded DataFrame to blob: band_prices.csv
